# <center>Scaling XGBoost Hyper-Parameter Optimization</center>

<img src="images/swarm.png" width="400"/>

# Motivation:

To reach highest performance in classification tasks (i.e., supervised learning ), it is best practice to build an ensemble of champion models. 

Each member of the ensemble is a winner of a search over many models of its kind with altered hyper-parameters.

In this notebook, we build a harness for running such a [hyper-parameter] search to demonstrate the accuracy benefits while exploring performance as we scale within and accross GPU nodes.

# Library Imports:

In [283]:
import ipyvolume as ipv
import matplotlib.pyplot as plt

import numpy as np; import pandas as pd; import cudf
import cuml; import xgboost; from xgboost import plot_tree
import warnings; warnings.filterwarnings('ignore')

import time; import copy 

import data_utils
import swarm
import visualization as viz

# reload library modules/code without a kernel restart
import importlib; importlib.reload( swarm ); importlib.reload( data_utils ); importlib.reload( viz);

# Dataset:

<center> In this notebook you can try different hyper-parameter search methods using synthetic or real data. </center>


<img src="images/datasets.png" width="800"/>

# [ Optional ] Synthetic Dataset Demo

In [149]:
exploreSyntheticDataFlag = False
if exploreSyntheticDataFlag:
    viz.visualize_synthetic_data_variants('whirl', nSamples = 10000, sdevScales = [ .3, .3, .3], nCoils = [2, 4, 6, 12])
    viz.visualize_synthetic_data_variants('helix', nSamples = 10000, sdevScales = [ .1, .1, .1], nCoils = [2, 4, 6, 12])

# Import / Generate Data

In [ ]:
cupy.asfortranarray()

In [312]:
# dataset = data_utils.Dataset('fashion-mnist')
# dataset = data_utils.Dataset('airline', nSamples = 1000000)
dataset = data_utils.Dataset('synthetic', coilType = 'whirl', coilDensity=15, nSamples=1000000, shuffleFlag=True)

# ETL [ split & normalize ]

# Visualize

In [285]:
dataset.trainData.shape, dataset.testData.shape

AttributeError: 'Dataset' object has no attribute 'trainData'

In [313]:
viz.plot_data( dataset.data, dataset.labels, dataset.datasetName )

plotting SYNTHETIC dataset, original shape: (1000000, 3)
 > plotting subset of 100000 samples -- 10.00% of total, adjust via maxSamplesToPlot 


In [314]:
def assign_middle_to_test_set ( data, labels, percentTrain = .8, targetDim ='x', percentTrainFromTestRegion = .015 ):
    
    dataSpan = ( data[targetDim].max() - data[targetDim].min() )
    dataMiddle = dataSpan/2.
    testDataSpan = dataSpan*(1-percentTrain)
    dataMiddle + testDataSpan/2.
    
    print(f'assigning middle [ along dim: {targetDim} ] as test set')
    testData = data[data[targetDim].ge( middle - extent )]
    testData = testData[testData[targetDim].le( middle + extent )]
    trainData = data[data[targetDim].ne( testData[targetDim]) ]
    testLabels = labels.iloc[testData.index]
    trainLabels = labels.iloc[trainData.index]

    samplesToSwap = int( trainData.shape[0] * percentTrainFromTestRegion )
    print(f'swapping {samplesToSwap} samples')
    if samplesToSwap > 0:
        # swap data
        for iColumn in trainData.columns:
            tempBuffer = trainData[iColumn][0:samplesToSwap].copy()
            trainData[iColumn][0:samplesToSwap] = testData[iColumn][0:samplesToSwap] 
            testData[iColumn][0:samplesToSwap] = tempBuffer
            
        # swap labels
        assert(labels.columns[0] == 'labels')
        tempBuffer = trainLabels['labels'][0:samplesToSwap].copy()
        trainLabels['labels'][0:samplesToSwap] = testLabels['labels'][0:samplesToSwap] 
        testLabels['labels'][0:samplesToSwap] = tempBuffer            

    return trainData, trainLabels, testData, testLabels  

In [315]:
%%time
trainData, trainLabels, testData, testLabels = assign_middle_to_test_set( dataset.data, dataset.labels )

assigning middle [ along dim: x ] as test set
swapping 11256 samples
CPU times: user 93.3 ms, sys: 36.4 ms, total: 130 ms
Wall time: 131 ms


In [316]:
trainData.shape

(750444, 3)

In [ ]:
creating a problem worthy of HPO
show typical
run model

show hard
run model

hpo


In [290]:
ipv.figure(width=800, height=800)


#xyz = trainData.as_matrix()
#ipv.scatter(xyz[:,0], xyz[:,1], xyz[:,2], color='#babab', marker = 'sphere', size = 1.25)

xyz = trainData[trainLabels.labels.eq(1)].as_matrix()
ipv.scatter(xyz[:,0], xyz[:,1], xyz[:,2], color='#8735fb', marker = 'sphere', size = 1)

xyz = trainData[trainLabels.labels.eq(0)].as_matrix()
ipv.scatter(xyz[:,0], xyz[:,1], xyz[:,2], color='green', marker = 'sphere', size = 1)

xyz = testData.as_matrix()
ipv.scatter(xyz[:,0], xyz[:,1], xyz[:,2], color='lightgray', marker = 'sphere', size = 2.75)


#ipv.pylab.squarelim()
#ipv.ylim(-12,12)
#ipv.zlim(-12,12)
ipv.xlim( ipv.gcf().xlim[0], ipv.gcf().xlim[1])
ipv.ylim( ipv.gcf().ylim[0]-5, ipv.gcf().ylim[1]+5)
ipv.zlim( ipv.gcf().zlim[0]-5, ipv.gcf().zlim[1]+5)
ipv.show()

In [320]:
%%time
dd = trainData.iloc[::10,:]
dl = trainLabels.iloc[::10]
l1 = dd[dl.labels.eq(1)].as_matrix()
l2 = dd[dl.labels.eq(0)].as_matrix()

CPU times: user 1.51 s, sys: 26.1 ms, total: 1.54 s
Wall time: 1.54 s


In [323]:
%%time
dd = trainData
dl = trainLabels
l1 = dd[dl.labels.eq(1)].as_matrix()
l2 = dd[dl.labels.eq(0)].as_matrix()

CPU times: user 36.4 ms, sys: 7.93 ms, total: 44.3 ms
Wall time: 45.7 ms


In [324]:
l2.shape

(375417, 3)

In [296]:
from ipywidgets import FloatSlider, ColorPicker, VBox, jslink

In [305]:
ipv.figure(width=800, height=800)
#xyz = trainData.as_matrix()
#ipv.scatter(xyz[:,0], xyz[:,1], xyz[:,2], color='#babab', marker = 'sphere', size = 1.25)


xyz = trainData[trainLabels.labels.eq(1)].as_matrix()
ipv.scatter(xyz[:,0], xyz[:,1], xyz[:,2], color='#8735fb', marker = 'sphere', size = 1)

xyz = trainData[trainLabels.labels.eq(0)].as_matrix()
ipv.scatter(xyz[:,0], xyz[:,1], xyz[:,2], color='green', marker = 'sphere', size = 1)

xyz = testData.as_matrix()
testDataScatter = ipv.scatter( xyz[:,0], xyz[:,1], xyz[:,2], color='lightgray', marker = 'sphere', size = 1, 
                               selected=np.arange(xyz.shape[0]), size_selected=1, color_selected='lightgray')

size_selected = FloatSlider(min=0, max=5, step=0.1)
jslink((testDataScatter, 'size_selected'), (size_selected, 'value'))
color_selected = ColorPicker()
jslink((testDataScatter, 'color_selected'), (color_selected, 'value'))
#ipv.pylab.squarelim()
#ipv.ylim(-12,12)
#ipv.zlim(-12,12)
ipv.xlim( ipv.gcf().xlim[0], ipv.gcf().xlim[1])
ipv.ylim( ipv.gcf().ylim[0]-5, ipv.gcf().ylim[1]+5)
ipv.zlim( ipv.gcf().zlim[0]-5, ipv.gcf().zlim[1]+5)
#ipv.show()
VBox([ipv.gcc(), size_selected, color_selected])

In [294]:
xz =ipv.gcf()

In [175]:
ipv.ylim( ipv.gcf().ylim[0]-5, ipv.gcf().ylim[1]+5) 

(-12.0, 12.0)

In [17]:
viz.plot_data( dataset.data.iloc[0::2,:], dataset.labels.iloc[0::2,:], dataset.datasetName )

plotting SYNTHETIC dataset, original shape: (5000, 3)


# Define Compute Cluster

In [ ]:
import dask
from dask import delayed
from dask_cuda import LocalCUDACluster

from dask.distributed import Client
from dask.distributed import as_completed
from dask.distributed import worker

In [ ]:
cluster = LocalCUDACluster( ip = '', n_workers = 4)

In [ ]:
client = Client( cluster, asynchronous = True)

In [ ]:
client

# Define HPO XGBoost Search Ranges

In [ ]:
paramRanges = { 0: ['max_depth', 3, 20, 'int'],
                1: ['learning_rate', .001, 1, 'float'],
                2: ['gamma', 0, 2, 'float'] }

# Search Strategies

&nbsp;

| method name | &nbsp;&nbsp;&nbsp; performance | &nbsp;&nbsp;&nbsp; search duration  |
|-----------------------|-----------------|------------------|
| random-search         | &nbsp;&nbsp;&nbsp; worst | &nbsp;&nbsp;&nbsp; slow    |
| particle-search [1]      | &nbsp;&nbsp;&nbsp; good  | &nbsp;&nbsp;&nbsp; fast    |
| async-particle-search | &nbsp;&nbsp;&nbsp; best  | &nbsp;&nbsp;&nbsp; fastest |

<center>[1] https://en.wikipedia.org/wiki/Particle_swarm_optimization#Algorithm</center>

# Key Params [ nParticles & nEpochs ]

# Sync vs Async [ Dask Task Stream ]

<img src='images/sync_vs_async.png' width='1000px'>

# Monitoring Perf [ <- ]
You'll have need to have launched the container w Port Open
Connect via [ the http:// is important]

# Single Run

In [ ]:
# fixed parameters
nTrees = 100
paramsGPU = { 'tree_method': 'gpu_hist',
              'max_depth': 6,
              'objective': 'binary:hinge'
            }

startTime = time.time()

trainDMatrix = xgboost.DMatrix( data = dataset.trainData, label = dataset.trainLabels )
trainedModelGPU = xgboost.train( dtrain = trainDMatrix, params = paramsGPU, num_boost_round = nTrees )

testDMatrix = xgboost.DMatrix( data = dataset.testData, label = dataset.testLabels )
predictionsGPU = trainedModelGPU.predict( testDMatrix ).astype(int)

trainAccuracy = 1 - float( trainedModelGPU.eval(trainDMatrix).split(':')[1] )
testAccuracy = 1 - float( trainedModelGPU.eval(testDMatrix).split(':')[1] )   

elapsedTime = time.time() - startTime

In [ ]:
print(f'{paramsGPU}, {nTrees}')
print(f'train accuracy: {trainAccuracy} \ntest accuracy: {testAccuracy} \ntrained in {elapsedTime:0.2f} seconds')

# Run HPO - <font color = '#ffb500'> Synchronous Swarm </font>

In [ ]:
syncSwarm = swarm.SyncSwarm( client, dataset, paramRanges, nParticles = 16, nEpochs = 10 )

In [ ]:
syncSwarm.run_search()

## Visualize <font color='#ffb500'> Synchronous Swarm </font>

In [ ]:
viz.plot_particle_evals( syncSwarm )

In [ ]:
viz.viz_particle_trails( syncSwarm, topN = 5 )

In [ ]:
viz.viz_swarm( syncSwarm, syncSwarm.paramRanges)

# Run HPO - <font color='#7400ff'> Asynchronous Swarm </font>

In [ ]:
asyncSwarm = swarm.AsyncSwarm( client, dataset, paramRanges, nParticles = 1, nEpochs = 1 )

In [ ]:
asyncSwarm.run_search()

## Visualize <font color='#7400ff'> Asynchronous Swarm </font>

In [ ]:
viz.plot_particle_evals( asyncSwarm )

In [ ]:
viz.viz_particle_trails( asyncSwarm, topN = 5 )

In [ ]:
viz.viz_swarm( asyncSwarm, asyncSwarm.paramRanges )

# Run HPO - <font color='#666666'> Random Search </font>

In [ ]:
randomAsyncSwarm = swarm.RandomSearchAsync ( client, dataset, paramRanges, nEpochs = 5 )

In [ ]:
randomAsyncSwarm.run_search()

## Visualize <font color='#666666'> Random Swarm </font>

In [ ]:
viz.plot_particle_evals( randomAsyncSwarm )

In [ ]:
viz.viz_particle_trails( randomAsyncSwarm, topN = 0 )

In [ ]:
viz.viz_swarm( randomAsyncSwarm, randomAsyncSwarm.paramRanges )

# Determine best swarm

In [ ]:
if syncSwarm.globalBest['accuracy'] > asyncSwarm.globalBest['accuracy']:
    swarm = syncSwarm
else:
    swarm = asyncSwarm

# Train XGBoost model with best params

In [ ]:
bestParams = {
    'tree_method': 'gpu_hist',
    'random_state': 0, 
    'max_depth': int(swarm.globalBest['params'][0]),
    'learning_rate': swarm.globalBest['params'][1],
    'gamma': swarm.globalBest['params'][2]
}
    
bestParams['objective'] = dataset.trainObjective[0]
if dataset.trainObjective[1] is not None: 
    bestParams['num_class'] = dataset.trainObjective[1]

In [ ]:
%%time
trainDMatrix = xgboost.DMatrix( data = dataset.trainData, label = dataset.trainLabels )
testDMatrix = xgboost.DMatrix( data = dataset.testData, label = dataset.testLabels )
trainedModelGPU = xgboost.train( dtrain = trainDMatrix, evals = [(testDMatrix, 'test')], params = bestParams,
                                 num_boost_round = swarm.globalBest['nTrees'], verbose_eval=False)

In [ ]:
# save best model
trainedModelGPU.save_model('xgb.model.hpo')

In [ ]:
from cuml import ForestInference

In [ ]:
?ForestInference.load

In [ ]:
dataset.trainOb

In [ ]:
fm = ForestInference.load( filename='xgb.model.hpo')

In [ ]:
fm = ForestInference.load( filename='xgb.model.hpo',
                           algo='BATCH_TREE_REORG',
                           output_class=True,
                           threshold=0,
                           model_type='xgboost' )

In [ ]:
type( dataset.testData )

In [ ]:
test = dataset.testData.as_gpu_matrix(order='C')

In [ ]:
for iColumn in dataset.testData.columns:    
    dataset.testData[iColumn] = dataset.testData[iColumn].astype('float32')

In [ ]:
dataset.testData.dtypes

In [ ]:
%%time
trainedModelGPU.predict( testDMatrix )

In [ ]:
%%time
filPredictions = fm.predict ( dataset.testData, 
                              algo='BATCH_TREE_REORG', 
                              output_class=True, 
                              threshold=0, 
                              model_type='xgboost' )

In [ ]:
cudf.Series(filPredictions)

In [ ]:
%%time
# perform prediction on the model loaded from path
fil_preds = fm.predict()

# Inference with trained model on test data

In [ ]:
%%time
predictions = trainedModelGPU.eval(testDMatrix)

In [ ]:
testDataPerf = 1 - float( predictions.split(':')[1] )
print(f'accuracy: {testDataPerf}')

# Summary

# Scale Up Results [ DGX-2 ]

<img src='images/synthetic_async.png'></img>

Async Scaling > Sync Scaling > Random Search

# References

In [ ]:
# https://www.osgeo.cn/matplotlib/gallery/lines_bars_and_markers/timeline.html

# Future Work / Extensions

larger than single GPU memory datasets - dask_cudf + [ dask_xgboost or xgboost.dask ]

1. [ Generate a classification dataset on GPU ](#data-load) (e.g., double helix, unwinding helix/whirl )

2. [ ETL - process/prepare data for model training ](#ETL) (e.g., scale, split, augment )   
    
3. [ Define HPO Strategy ](#define-hpo)

4. [ Create Compute Cluster ](#compute-cluster)
   > LocalCUDACluster or KubeCluster
      
5. [ Define Seach ](#define-search)

6. [ Run ASYNC Particle Swarm ](#run-async-PSO)

7. [ Run Classic Particle Swarm ](#run-classic-PSO)

8. [ Run Random Search Baseline ](#run-random-search)

9. [ Summary ](#summary)

# User Choices:

The user is able to make several key choices in running this notebook. They are as follows:

1. [ Dataset ]()
2. [ Compute Scaling Strategy - Scale-Up, Scale-Out ]()
3. [ XGBoost Parameter Search Range ]()

4. [ Particle Swarm Type ]()
   * Synchronous
   * Asynchronous
   * Random Search